# Dynamic Risk Assessment Network Analysis

## Overview
This notebook implements Dynamic Risk Assessment methodology to identify and visualize risk networks with special focus on:

- **"Must Go Right" Risks (Green Arrows)**: Critical risks that act as protective barriers or foundational elements. When these succeed/remain stable, they prevent other risks from being triggered.

- **"Can't Go Wrong" Risks (Red Arrows)**: High-impact risks that should be avoided at all costs, as their failure would trigger cascading effects throughout the risk network.

## Methodology
The analysis uses network theory, centrality measures, and impact propagation modeling to classify risks based on:
1. **Likelihood** - Probability of occurrence
2. **Impact** - Severity of consequences  
3. **Velocity** - Speed of impact propagation
4. **Connectivity** - How risks interconnect and trigger contagion effects


In [7]:
# Install required libraries
!pip install networkx matplotlib pandas numpy seaborn plotly ipywidgets

# Import libraries
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib.patches import FancyBboxPatch
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print("📊 Ready for DRA Network Analysis")


✅ Libraries imported successfully!
📊 Ready for DRA Network Analysis


In [8]:
class DRARiskNetwork:
    """
    Dynamic Risk Assessment Network Analysis Class
    
    This class implements the DRA methodology to identify and analyze risk networks,
    specifically focusing on "Must Go Right" and "Can't Go Wrong" risk classifications.
    
    Key Concepts:
    - "Must Go Right" risks: Critical enablers/protectors that must succeed to prevent cascading failures
    - "Can't Go Wrong" risks: Critical amplifiers/triggers that cause cascading failures when they occur
    """
    
    def __init__(self):
        self.G = nx.DiGraph()  # Directed graph for risk relationships
        self.risk_data = {}    # Store risk attributes
        self.classifications = {}  # Store risk classifications
        
    def add_risk(self, risk_id, name, likelihood, impact, velocity=1.0, category="Operational"):
        """
        Add a risk to the network
        
        Parameters:
        - risk_id: Unique identifier for the risk
        - name: Human-readable risk name
        - likelihood: Probability of occurrence (0-1)
        - impact: Severity of consequences (0-10)
        - velocity: Speed of impact propagation (0-10)
        - category: Risk category (Operational, Financial, Strategic, etc.)
        """
        self.G.add_node(risk_id)
        self.risk_data[risk_id] = {
            'name': name,
            'likelihood': likelihood,
            'impact': impact,
            'velocity': velocity,
            'category': category,
            'risk_score': likelihood * impact * velocity
        }
        
    def add_risk_relationship(self, source_risk, target_risk, strength=1.0, relationship_type="triggers"):
        """
        Add a relationship between risks
        
        Parameters:
        - source_risk: Risk that triggers or influences
        - target_risk: Risk that is triggered or influenced
        - strength: Strength of the relationship (0-1)
        - relationship_type: Type of relationship (triggers, mitigates, amplifies, enables)
        """
        self.G.add_edge(source_risk, target_risk, 
                       strength=strength, 
                       relationship_type=relationship_type)
    
    def calculate_centrality_measures(self):
        """Calculate various centrality measures for risk classification"""
        centrality_measures = {}
        
        # Degree centrality (incoming connections)
        centrality_measures['in_degree'] = nx.in_degree_centrality(self.G)
        centrality_measures['out_degree'] = nx.out_degree_centrality(self.G)
        
        # Betweenness centrality (bridge between risks)
        centrality_measures['betweenness'] = nx.betweenness_centrality(self.G)
        
        # Closeness centrality (proximity to other risks)
        centrality_measures['closeness'] = nx.closeness_centrality(self.G)
        
        # PageRank (importance based on connections)
        centrality_measures['pagerank'] = nx.pagerank(self.G)
        
        return centrality_measures
    
    def classify_risks(self):
        """
        Classify risks as "Must Go Right", "Can't Go Wrong", or "Standard"
        based on DRA methodology
        
        Must Go Right: High protective value, low likelihood, high impact when they fail
        Can't Go Wrong: High cascade potential, high likelihood, high impact when they occur
        """
        centrality = self.calculate_centrality_measures()
        
        for risk_id in self.G.nodes():
            risk_info = self.risk_data[risk_id]
            
            # Calculate protective score (Must Go Right potential)
            # High incoming connections + low likelihood + high impact = protective barrier
            protective_score = (
                centrality['in_degree'][risk_id] * 0.4 +  # Many risks depend on this
                (1 - risk_info['likelihood']) * 0.3 +    # Low likelihood (stable)
                risk_info['impact'] * 0.2 +               # High impact when it fails
                centrality['betweenness'][risk_id] * 0.1  # Bridge role
            )
            
            # Calculate cascade score (Can't Go Wrong potential)
            # High outgoing connections + high likelihood + high velocity = cascade trigger
            cascade_score = (
                centrality['out_degree'][risk_id] * 0.4 +  # Triggers many other risks
                risk_info['likelihood'] * 0.3 +            # High likelihood of occurring
                risk_info['velocity'] * 0.2 +              # High velocity of impact
                centrality['pagerank'][risk_id] * 0.1      # High importance in network
            )
            
            # Classification logic based on DRA methodology
            if protective_score > 0.7 and risk_info['impact'] > 7:
                classification = "Must Go Right"
                color = "green"
                priority = "Critical"
                description = "Critical enabler/protector - must succeed to prevent cascading failures"
            elif cascade_score > 0.7 and risk_info['likelihood'] > 0.4:
                classification = "Can't Go Wrong"
                color = "red"
                priority = "Critical"
                description = "Critical amplifier/trigger - causes cascading failures when it occurs"
            elif risk_info['risk_score'] > 6:
                classification = "High Priority"
                color = "orange"
                priority = "High"
                description = "Important risk requiring attention"
            else:
                classification = "Standard"
                color = "blue"
                priority = "Medium"
                description = "Standard risk with moderate impact"
            
            self.classifications[risk_id] = {
                'classification': classification,
                'color': color,
                'priority': priority,
                'description': description,
                'protective_score': protective_score,
                'cascade_score': cascade_score
            }
    
    def get_risk_summary(self):
        """Generate a summary of risk classifications"""
        if not self.classifications:
            self.classify_risks()
        
        summary = {}
        for risk_id, classification in self.classifications.items():
            risk_name = self.risk_data[risk_id]['name']
            class_type = classification['classification']
            
            if class_type not in summary:
                summary[class_type] = []
            
            summary[class_type].append({
                'risk_id': risk_id,
                'name': risk_name,
                'risk_score': self.risk_data[risk_id]['risk_score'],
                'protective_score': classification['protective_score'],
                'cascade_score': classification['cascade_score'],
                'description': classification['description']
            })
        
        return summary

# Initialize the DRA Risk Network
dra_network = DRARiskNetwork()
print("🎯 DRA Risk Network initialized successfully!")
print("📋 Key Concepts:")
print("   🟢 Must Go Right: Critical enablers/protectors that must succeed")
print("   🔴 Can't Go Wrong: Critical amplifiers/triggers that cause cascading failures")


🎯 DRA Risk Network initialized successfully!
📋 Key Concepts:
   🟢 Must Go Right: Critical enablers/protectors that must succeed
   🔴 Can't Go Wrong: Critical amplifiers/triggers that cause cascading failures


In [9]:
# Create a climate-focused risk network based on cas_m1 ass_v4.ipynb factors
# This represents Nestlé's climate risk scenario with proper DRA classifications

# Add risks to the network - Climate-focused risks
risks = [
    # Must Go Right risks (critical enablers/protectors for climate resilience)
    ("Climate_Strategy", "Climate Strategy & Governance", 0.15, 9.5, 8.5, "Strategic"),
    ("ESG_Compliance", "ESG Compliance Framework", 0.2, 9.0, 7.5, "Regulatory"),
    ("Supply_Chain_Resilience", "Supply Chain Resilience Systems", 0.18, 8.5, 7.0, "Operational"),
    ("Energy_Efficiency", "Energy Efficiency Programs", 0.1, 8.0, 6.5, "Operational"),
    ("Carbon_Offsetting", "Carbon Offsetting Programs", 0.12, 7.5, 6.0, "Environmental"),
    
    # Can't Go Wrong risks (critical amplifiers/triggers for climate impacts)
    ("Carbon_Tax_Increase", "Carbon Tax Escalation", 0.45, 9.8, 9.5, "Regulatory"),
    ("Supply_Chain_Disruption", "Supply Chain Climate Disruption", 0.55, 8.5, 8.8, "Operational"),
    ("Raw_Material_Price_Spike", "Raw Material Price Volatility", 0.4, 9.2, 9.0, "Market"),
    ("Energy_Cost_Surge", "Energy Cost Escalation", 0.5, 9.0, 9.2, "Operational"),
    ("Regulatory_Clampdown", "Climate Regulatory Tightening", 0.35, 9.5, 8.5, "Regulatory"),
    
    # High Priority risks (significant climate-related impacts)
    ("Reputation_Damage", "Climate Reputation Risk", 0.3, 7.8, 7.2, "Strategic"),
    ("Consumer_Pressure", "Consumer Climate Pressure", 0.4, 7.5, 6.8, "Market"),
    ("Investor_Divestment", "Investor Climate Divestment", 0.25, 8.0, 7.5, "Financial"),
    ("Technology_Disruption", "Climate Tech Disruption", 0.35, 7.0, 6.5, "Technology"),
    
    # Standard risks (baseline climate considerations)
    ("Weather_Volatility", "Extreme Weather Events", 0.6, 6.5, 5.5, "Environmental"),
    ("Competitor_Advantage", "Competitor Climate Advantage", 0.7, 6.8, 5.8, "Strategic"),
    ("Employee_Retention", "Climate Talent Retention", 0.5, 6.0, 5.2, "Human Resources"),
    ("Insurance_Costs", "Climate Insurance Premiums", 0.4, 6.5, 5.8, "Financial")
]

# Add risks to the network
for risk_id, name, likelihood, impact, velocity, category in risks:
    dra_network.add_risk(risk_id, name, likelihood, impact, velocity, category)

# Define risk relationships (how climate risks influence each other)
relationships = [
    # Must Go Right risks protect against Can't Go Wrong risks
    ("Climate_Strategy", "Carbon_Tax_Increase", 0.8, "mitigates"),
    ("Climate_Strategy", "Regulatory_Clampdown", 0.7, "mitigates"),
    ("Climate_Strategy", "Reputation_Damage", 0.6, "mitigates"),
    ("ESG_Compliance", "Regulatory_Clampdown", 0.9, "mitigates"),
    ("ESG_Compliance", "Reputation_Damage", 0.8, "mitigates"),
    ("ESG_Compliance", "Investor_Divestment", 0.7, "mitigates"),
    ("Supply_Chain_Resilience", "Supply_Chain_Disruption", 0.9, "mitigates"),
    ("Supply_Chain_Resilience", "Raw_Material_Price_Spike", 0.6, "mitigates"),
    ("Energy_Efficiency", "Energy_Cost_Surge", 0.8, "mitigates"),
    ("Energy_Efficiency", "Insurance_Costs", 0.6, "mitigates"),
    ("Carbon_Offsetting", "Carbon_Tax_Increase", 0.6, "mitigates"),
    ("Carbon_Offsetting", "Reputation_Damage", 0.5, "mitigates"),
    
    # Can't Go Wrong risks trigger multiple cascading risks
    ("Carbon_Tax_Increase", "Raw_Material_Price_Spike", 0.7, "triggers"),
    ("Carbon_Tax_Increase", "Energy_Cost_Surge", 0.8, "triggers"),
    ("Carbon_Tax_Increase", "Consumer_Pressure", 0.6, "triggers"),
    ("Carbon_Tax_Increase", "Insurance_Costs", 0.5, "triggers"),
    
    ("Supply_Chain_Disruption", "Raw_Material_Price_Spike", 0.9, "triggers"),
    ("Supply_Chain_Disruption", "Reputation_Damage", 0.6, "triggers"),
    ("Supply_Chain_Disruption", "Consumer_Pressure", 0.7, "triggers"),
    ("Supply_Chain_Disruption", "Technology_Disruption", 0.5, "triggers"),
    
    ("Raw_Material_Price_Spike", "Consumer_Pressure", 0.7, "triggers"),
    ("Raw_Material_Price_Spike", "Reputation_Damage", 0.5, "triggers"),
    ("Raw_Material_Price_Spike", "Competitor_Advantage", 0.6, "triggers"),
    
    ("Energy_Cost_Surge", "Technology_Disruption", 0.5, "triggers"),
    ("Energy_Cost_Surge", "Insurance_Costs", 0.7, "triggers"),
    ("Energy_Cost_Surge", "Competitor_Advantage", 0.4, "triggers"),
    
    ("Regulatory_Clampdown", "Investor_Divestment", 0.6, "triggers"),
    ("Regulatory_Clampdown", "Reputation_Damage", 0.5, "triggers"),
    ("Regulatory_Clampdown", "Consumer_Pressure", 0.4, "triggers"),
    
    # High Priority risks amplify each other
    ("Reputation_Damage", "Consumer_Pressure", 0.8, "amplifies"),
    ("Reputation_Damage", "Investor_Divestment", 0.7, "amplifies"),
    ("Consumer_Pressure", "Investor_Divestment", 0.7, "amplifies"),
    ("Consumer_Pressure", "Reputation_Damage", 0.6, "amplifies"),
    ("Investor_Divestment", "Reputation_Damage", 0.6, "amplifies"),
    ("Investor_Divestment", "Consumer_Pressure", 0.5, "amplifies"),
    
    # Standard risks that enable or trigger other risks
    ("Weather_Volatility", "Supply_Chain_Disruption", 0.8, "triggers"),
    ("Weather_Volatility", "Raw_Material_Price_Spike", 0.6, "triggers"),
    ("Weather_Volatility", "Insurance_Costs", 0.7, "triggers"),
    ("Weather_Volatility", "Energy_Cost_Surge", 0.4, "triggers"),
    
    ("Competitor_Advantage", "Consumer_Pressure", 0.5, "triggers"),
    ("Competitor_Advantage", "Reputation_Damage", 0.4, "triggers"),
    ("Competitor_Advantage", "Employee_Retention", 0.6, "triggers"),
    
    ("Technology_Disruption", "Competitor_Advantage", 0.7, "enables"),
    ("Technology_Disruption", "Energy_Cost_Surge", 0.5, "triggers"),
    ("Technology_Disruption", "Supply_Chain_Disruption", 0.4, "triggers"),
    
    ("Employee_Retention", "Climate_Strategy", 0.5, "enables"),
    ("Employee_Retention", "ESG_Compliance", 0.6, "enables"),
    ("Employee_Retention", "Technology_Disruption", 0.4, "triggers"),
    
    ("Insurance_Costs", "Energy_Cost_Surge", 0.4, "triggers"),
    ("Insurance_Costs", "Raw_Material_Price_Spike", 0.3, "triggers"),
    ("Insurance_Costs", "Weather_Volatility", 0.6, "triggers"),
    
    # Cross-category enabling relationships
    ("Climate_Strategy", "Weather_Volatility", 0.5, "mitigates"),
    ("Climate_Strategy", "Employee_Retention", 0.6, "enables"),
    ("ESG_Compliance", "Employee_Retention", 0.7, "enables"),
    ("Supply_Chain_Resilience", "Weather_Volatility", 0.6, "mitigates"),
    ("Energy_Efficiency", "Technology_Disruption", 0.4, "mitigates"),
    ("Carbon_Offsetting", "Consumer_Pressure", 0.3, "mitigates")
]

# Add relationships to the network
for source, target, strength, rel_type in relationships:
    dra_network.add_risk_relationship(source, target, strength, rel_type)

# Classify risks
dra_network.classify_risks()

print("📊 Climate-focused risk network created with", len(risks), "risks and", len(relationships), "relationships")
print("🔍 Risk classification completed!")
print("\n🌍 Climate Risk DRA Methodology Applied:")
print("   • Must Go Right risks are critical climate resilience enablers/protectors")
print("   • Can't Go Wrong risks are critical climate impact amplifiers/triggers")
print("   • Network analysis identifies climate risk cascading failure patterns")
print("   • Based on Nestlé's climate P&L impact factors from cas_m1 ass_v4.ipynb")


ModuleNotFoundError: No module named 'scipy'

In [ ]:
# Display risk classification summary
summary = dra_network.get_risk_summary()

print("🌍 DRA Climate Risk Classification Summary")
print("=" * 55)

for classification_type, risks in summary.items():
    print(f"\n📋 {classification_type} Risks:")
    print("-" * 30)
    
    for risk in risks:
        print(f"• {risk['name']}")
        print(f"  Risk Score: {risk['risk_score']:.2f}")
        print(f"  Protective Score: {risk['protective_score']:.2f}")
        print(f"  Cascade Score: {risk['cascade_score']:.2f}")
        print()

# Create a detailed risk analysis DataFrame
risk_analysis = []
for risk_id in dra_network.G.nodes():
    risk_info = dra_network.risk_data[risk_id]
    classification = dra_network.classifications[risk_id]
    
    risk_analysis.append({
        'Risk ID': risk_id,
        'Risk Name': risk_info['name'],
        'Category': risk_info['category'],
        'Likelihood': risk_info['likelihood'],
        'Impact': risk_info['impact'],
        'Velocity': risk_info['velocity'],
        'Risk Score': risk_info['risk_score'],
        'Classification': classification['classification'],
        'Priority': classification['priority'],
        'Protective Score': classification['protective_score'],
        'Cascade Score': classification['cascade_score']
    })

df_risks = pd.DataFrame(risk_analysis)
df_risks = df_risks.sort_values('Risk Score', ascending=False)

print("📊 Detailed Risk Analysis:")
print(df_risks.to_string(index=False))


In [ ]:
def visualize_dra_network(dra_network, figsize=(15, 12)):
    """
    Create a comprehensive visualization of the DRA risk network
    with color-coded nodes and arrows based on risk classifications
    """
    
    # Set up the plot
    fig, ax = plt.subplots(figsize=figsize)
    
    # Calculate layout
    pos = nx.spring_layout(dra_network.G, k=3, iterations=50, seed=42)
    
    # Define colors for different classifications
    color_map = {
        "Must Go Right": "#2E8B57",      # Sea Green
        "Can't Go Wrong": "#DC143C",     # Crimson Red
        "High Priority": "#FF8C00",     # Dark Orange
        "Standard": "#4682B4"            # Steel Blue
    }
    
    # Get node colors
    node_colors = []
    node_sizes = []
    node_labels = {}
    
    for node in dra_network.G.nodes():
        classification = dra_network.classifications[node]['classification']
        node_colors.append(color_map[classification])
        
        # Size based on risk score
        risk_score = dra_network.risk_data[node]['risk_score']
        node_sizes.append(risk_score * 200 + 300)
        
        # Labels
        node_labels[node] = dra_network.risk_data[node]['name']
    
    # Draw edges with different styles based on relationship type
    edge_colors = []
    edge_widths = []
    
    for edge in dra_network.G.edges():
        edge_data = dra_network.G[edge[0]][edge[1]]
        rel_type = edge_data['relationship_type']
        strength = edge_data['strength']
        
        if rel_type == "triggers":
            edge_colors.append("#FF6B6B")  # Red for triggers
            edge_widths.append(strength * 3)
        elif rel_type == "mitigates":
            edge_colors.append("#4ECDC4")  # Teal for mitigates
            edge_widths.append(strength * 2)
        elif rel_type == "amplifies":
            edge_colors.append("#FFE66D")  # Yellow for amplifies
            edge_widths.append(strength * 2.5)
        else:
            edge_colors.append("#95A5A6")  # Gray for other
            edge_widths.append(strength * 2)
    
    # Draw the network
    nx.draw_networkx_nodes(dra_network.G, pos, 
                          node_color=node_colors,
                          node_size=node_sizes,
                          alpha=0.8)
    
    nx.draw_networkx_edges(dra_network.G, pos,
                          edge_color=edge_colors,
                          width=edge_widths,
                          alpha=0.6,
                          arrows=True,
                          arrowsize=20,
                          arrowstyle='->')
    
    # Add labels
    nx.draw_networkx_labels(dra_network.G, pos, 
                           labels=node_labels,
                           font_size=8,
                           font_weight='bold')
    
    # Create legend
    legend_elements = []
    for classification, color in color_map.items():
        legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                         markerfacecolor=color, markersize=10, 
                                         label=classification))
    
    ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1))
    
    # Add title and formatting
    plt.title("Dynamic Risk Assessment Network - Climate Focus\n" + 
              "Green = Must Go Right | Red = Can't Go Wrong | Orange = High Priority | Blue = Standard",
              fontsize=14, fontweight='bold', pad=20)
    
    plt.axis('off')
    plt.tight_layout()
    
    return fig, ax

# Create the network visualization
fig, ax = visualize_dra_network(dra_network)
plt.show()

print("🎨 Network visualization created!")
print("📝 Legend:")
print("   🟢 Green nodes = Must Go Right risks (protective barriers)")
print("   🔴 Red nodes = Can't Go Wrong risks (high cascade potential)")
print("   🟠 Orange nodes = High Priority risks")
print("   🔵 Blue nodes = Standard risks")
print("   🔴 Red arrows = Triggers relationships")
print("   🔵 Teal arrows = Mitigates relationships")
print("   🟡 Yellow arrows = Amplifies relationships")


In [10]:
# Create interactive risk analysis dashboard
def create_risk_dashboard(dra_network):
    """Create an interactive dashboard showing risk metrics and classifications"""
    
    # Prepare data for visualizations
    risk_data = []
    for risk_id in dra_network.G.nodes():
        risk_info = dra_network.risk_data[risk_id]
        classification = dra_network.classifications[risk_id]
        
        risk_data.append({
            'Risk Name': risk_info['name'],
            'Category': risk_info['category'],
            'Likelihood': risk_info['likelihood'],
            'Impact': risk_info['impact'],
            'Velocity': risk_info['velocity'],
            'Risk Score': risk_info['risk_score'],
            'Classification': classification['classification'],
            'Protective Score': classification['protective_score'],
            'Cascade Score': classification['cascade_score']
        })
    
    df = pd.DataFrame(risk_data)
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Risk Score Distribution', 'Classification Breakdown', 
                       'Protective vs Cascade Scores', 'Risk Categories'),
        specs=[[{"type": "bar"}, {"type": "pie"}],
               [{"type": "scatter"}, {"type": "bar"}]]
    )
    
    # Color mapping
    color_mapping = {
        'Must Go Right': '#2E8B57',
        "Can't Go Wrong": '#DC143C', 
        'High Priority': '#FF8C00',
        'Standard': '#4682B4'
    }
    
    # 1. Risk Score Distribution
    bar_colors = [color_mapping[cls] for cls in df['Classification']]
    fig.add_trace(
        go.Bar(x=df['Risk Name'], y=df['Risk Score'], 
               marker_color=bar_colors,
               name='Risk Score'),
        row=1, col=1
    )
    
    # 2. Classification Breakdown
    classification_counts = df['Classification'].value_counts()
    fig.add_trace(
        go.Pie(labels=classification_counts.index, values=classification_counts.values,
               marker_colors=['#2E8B57', '#DC143C', '#FF8C00', '#4682B4']),
        row=1, col=2
    )
    
    # 3. Protective vs Cascade Scores
    scatter_colors = [color_mapping[cls] for cls in df['Classification']]
    fig.add_trace(
        go.Scatter(x=df['Protective Score'], y=df['Cascade Score'],
                  mode='markers+text',
                  text=df['Risk Name'],
                  textposition="top center",
                  marker=dict(size=df['Risk Score']*3,
                            color=scatter_colors),
                  name='Risk Positioning'),
        row=2, col=1
    )
    
    # 4. Risk Categories
    category_counts = df['Category'].value_counts()
    fig.add_trace(
        go.Bar(x=category_counts.index, y=category_counts.values,
               name='Category Count'),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        title_text="DRA Risk Analysis Dashboard",
        title_x=0.5,
        height=800,
        showlegend=False
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Risks", row=1, col=1)
    fig.update_yaxes(title_text="Risk Score", row=1, col=1)
    fig.update_xaxes(title_text="Protective Score", row=2, col=1)
    fig.update_yaxes(title_text="Cascade Score", row=2, col=1)
    fig.update_xaxes(title_text="Categories", row=2, col=2)
    fig.update_yaxes(title_text="Count", row=2, col=2)
    
    return fig

# Create and display the dashboard
dashboard = create_risk_dashboard(dra_network)
dashboard.show()

print("📊 Interactive risk analysis dashboard created!")
print("💡 Key Insights:")
print("   • Must Go Right risks have high protective scores and low likelihood")
print("   • Can't Go Wrong risks have high cascade scores and high likelihood")
print("   • The scatter plot shows risk positioning in the protective-cascade space")
print("   • Larger markers indicate higher risk scores")


KeyError: 'Climate_Strategy'

# 🌐 Create Online Shareable Dashboard
# Install Gradio for online dashboard sharing
!pip install gradio

import gradio as gr

def create_gradio_dashboard():
    """Create an interactive Gradio dashboard for the climate risk analysis"""
    
    def update_dashboard():
        # Get current risk data
        risk_data = []
        for risk_id in dra_network.G.nodes():
            risk_info = dra_network.risk_data[risk_id]
            classification = dra_network.classifications[risk_id]
            
            risk_data.append({
                'Risk Name': risk_info['name'],
                'Category': risk_info['category'],
                'Likelihood': risk_info['likelihood'],
                'Impact': risk_info['impact'],
                'Velocity': risk_info['velocity'],
                'Risk Score': risk_info['risk_score'],
                'Classification': classification['classification'],
                'Protective Score': classification['protective_score'],
                'Cascade Score': classification['cascade_score']
            })
        
        df = pd.DataFrame(risk_data)
        
        # Create the dashboard
        dashboard = create_risk_dashboard(dra_network)
        
        return dashboard, df
    
    # Create Gradio interface
    with gr.Blocks(title="KPMG DRA Climate Risk Dashboard", theme=gr.themes.Soft()) as interface:
        gr.Markdown("""
        # 🌍 KPMG Dynamic Risk Assessment - Climate Risk Analysis
        
        **Interactive Dashboard for Nestlé Climate Risk Network Analysis**
        
        This dashboard visualizes climate risks using KPMG's DRA methodology:
        - 🟢 **Must Go Right**: Critical climate resilience enablers/protectors
        - 🔴 **Can't Go Wrong**: Critical climate impact amplifiers/triggers
        - 🟠 **High Priority**: Significant climate-related impacts
        - 🔵 **Standard**: Baseline climate considerations
        """)
        
        with gr.Row():
            with gr.Column():
                gr.Markdown("### 📊 Risk Analysis Dashboard")
                dashboard_plot = gr.Plot()
                
            with gr.Column():
                gr.Markdown("### 📋 Risk Data Table")
                risk_table = gr.Dataframe(
                    headers=["Risk Name", "Category", "Likelihood", "Impact", 
                           "Velocity", "Risk Score", "Classification", 
                           "Protective Score", "Cascade Score"],
                    interactive=False
                )
        
        # Load initial data
        interface.load(update_dashboard, outputs=[dashboard_plot, risk_table])
    
    return interface

# Create and launch the dashboard
print("🚀 Creating online dashboard...")
dashboard_interface = create_gradio_dashboard()
print("🌐 Launching dashboard with public URL...")
print("📱 The dashboard will be mobile-responsive and accessible from anywhere!")
print("🔗 Copy the URL below to share with your team:")
print("-" * 60)

# Launch the dashboard and capture the URL
try:
    dashboard_interface.launch(share=True, debug=True, show_error=True, quiet=False)
except Exception as e:
    print(f"Error launching dashboard: {e}")
    print("Trying alternative launch method...")
    dashboard_interface.launch(share=True, debug=False, quiet=False)


## Usage Instructions

### How to Use This DRA Network Analysis Tool

1. **Define Your Risk Network**: 
   - Use `dra_network.add_risk()` to add risks with likelihood, impact, and velocity scores
   - Use `dra_network.add_risk_relationship()` to define how risks influence each other

2. **Run Analysis**:
   - The system automatically classifies risks as "Must Go Right", "Can't Go Wrong", "High Priority", or "Standard"
   - View the network visualization to see risk relationships and classifications

3. **Interpret Results**:
   - **Green nodes (Must Go Right)**: Critical protective barriers that prevent other risks
   - **Red nodes (Can't Go Wrong)**: High-impact risks that trigger cascading failures
   - **Orange nodes (High Priority)**: Important risks requiring attention
   - **Blue nodes (Standard)**: Regular risks with moderate impact

### Key Features

- **Network Visualization**: Interactive graph showing risk relationships
- **Risk Classification**: Automated identification of critical risk types
- **Centrality Analysis**: Measures of risk importance and connectivity
- **Impact Propagation**: Modeling of how risks cascade through the network
- **Interactive Dashboard**: Comprehensive analysis with multiple visualizations

### Next Steps

To customize this analysis for your specific organization:

1. Replace the sample risks with your actual risk inventory
2. Adjust the classification thresholds based on your risk appetite
3. Add industry-specific risk categories and relationships
4. Integrate with your existing risk management systems

This tool provides a foundation for implementing KPMG's Dynamic Risk Assessment methodology in your organization.
